In [5]:
!pip install keras_preprocessing




[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [7]:
TRAIN_DIR = 'images/train'
TEST_DIR= 'images/test'

In [8]:
def createdataframe(dir):
    image_paths=[]
    labels=[]
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_paths,labels


In [9]:
train=pd.DataFrame()
train['image'],train['label']=createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [10]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [11]:
test=pd.DataFrame()
test['image'],test['label']=createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [12]:
print(test)

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]


In [13]:
from tqdm.notebook import tqdm

In [14]:
!pip install Pillow



[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
def extract_features(images) :
    features=[]
    for image in tqdm(images):
        img=load_img(image,color_mode='grayscale')
        img=np.array(img)
        features.append(img)
    features=np.array(features)
    features=features.reshape(len(features),48,48,1)
    return features
        

In [18]:
train_features=extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [19]:
test_features=extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [20]:
x_train=train_features/255.0
x_test=test_features/255.0

In [25]:
!pip install scikit-learn


[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
from sklearn.preprocessing import LabelEncoder


In [27]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [28]:
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])

In [29]:
y_train=to_categorical(y_train,num_classes=7)
y_test=to_categorical(y_test,num_classes=7)

In [30]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()

# Convolutional Layers
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# Flatten
model.add(Flatten())

# Fully Connected Layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# Output Layer (7 classes)
model.add(Dense(7, activation='softmax'))


C:\Users\jatin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [31]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [32]:
model.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_test, y_test)
)

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 116s 508ms/step - accuracy: 0.2385 - loss: 1.8361 - val_accuracy: 0.2583 - val_loss: 1.8022
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 119s 526ms/step - accuracy: 0.2573 - loss: 1.7924 - val_accuracy: 0.3166 - val_loss: 1.6736
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 119s 526ms/step - accuracy: 0.3259 - loss: 1.6642 - val_accuracy: 0.4435 - val_loss: 1.4521
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 120s 531ms/step - accuracy: 0.4114 - loss: 1.5060 - val_accuracy: 0.4840 - val_loss: 1.3535
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 119s 528ms/step - accuracy: 0.4494 - loss: 1.4189 - val_accuracy: 0.5000 - val_loss: 1.2917
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 120s 529ms/step - accuracy: 0.4728 - loss: 1.3674 - val_accuracy: 0.5218 - val_loss: 1.2486
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 123s 544ms/step - accuracy: 0.4885 - loss: 1.3352 - val_accuracy: 0.5331 - val_loss: 1.2407
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 125s 553ms/step - accuracy: 

KeyboardInterrupt: 

In [34]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [35]:
from keras.models import model_from_json

In [36]:
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [37]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [38]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [39]:
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

original image is of sad


C:\Users\jatin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
model prediction is  sad
